In [56]:
from sqlalchemy import text, create_engine
import pandas as pd
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 

In [57]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [58]:
consultant_name_df = pd.read_excel('G:/Post Construction/metadata csvs/consultants.xlsx', encoding = 'utf-8-sig')
consultant_name_lst = consultant_name_df["consultant_name"].tolist()

In [59]:
with engine.connect() as conn: ## with context manager with, we don't have to close the connection everytime we run queries
    query1 = "SELECT * FROM pdfs;"
    df1 = pd.read_sql(query1, conn)

In [60]:
main_consultant_name = []
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\t1nipun\AppData\Local\Tesseract-OCR\tesseract.exe'
for file in df1['pdfName']:
    pages = convert_from_path("G:/Post Construction/PCMR_All/" + file + '.pdf', 500, last_page = 1)
    for page in pages:
        filename = file +'_Page1.jpg'
        page.save("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + filename, 'JPEG')
        text = str(((pytesseract.image_to_string(Image.open("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + filename))))).lower()
        consultant_name = []
        for name in consultant_name_lst:
            if name.lower() in text:
                consultant_name.append(name)
        main_consultant_name.append(consultant_name)

In [66]:
df1['consultant_name'] = main_consultant_name
df1

,pdfId,pdfName,pdfSize,filingId,date,totalPages,status,xmlContent,application_id,submitter,company,consultant_name
0,337444,A0L2F6 - 002 Ekwan Post Construction Monitorin...,0.05,337440,2004-10-06 14:46:30,11,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",268876,EnCana Ekwan Pipeline Inc.,EnCana Ekwan Pipeline Inc.,[AXYS Environmental Consulting Ltd]
1,340413,A0L3K4 - Kwoen Facilities and Re-injection Ext...,1.02,340412,2004-11-15 11:07:40,19,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",139307,"Westcoast Energy Inc., a Duke Energy company",Westcoast Energy Inc.,[]
2,399963,A0T5H7 - Kwoen Re-injection Pipeline - 3rd Yea...,1.60,399962,2006-03-06 16:18:03,25,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",139307,"Westcoast Energy Inc., carrying on business as...",Westcoast Energy Inc.,[]
3,408901,A0U3V0 - 002 Post Construction Monitoring - Au...,0.54,409162,2006-05-30 09:23:58,29,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",268876,Alan K. Haskey,EnCana Ekwan Pipeline Inc.,[AXYS Environmental Consulting Ltd]
4,476639,B-14R Appendix MPLA-SAPL IR 43 b) - TERA Post ...,7.16,476698,2007-09-10 12:30:32,301,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",465178,Enbridge Pipelines Inc.,Enbridge Pipelines Inc.,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
244,3751108,A97686-2 Attachment 1 - Post Construction Envi...,7.28,3751106,2019-01-28 16:06:17,113,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",893373,Enbridge Pipelines Inc.,Enbridge Pipelines Inc.,[]
245,3751134,A97703-2 NWML_Cranberry Section_Year 5_PCMR_Ja...,4.47,3751132,2019-01-29 14:04:08,58,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",737909,NOVA Gas Transmission Ltd.,NOVA Gas Transmission Ltd.,[]
246,3751136,A97703-4 NWML_Timberwolf Section_Year 5_PCMR_J...,5.97,3751132,2019-01-29 14:04:19,104,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",737909,NOVA Gas Transmission Ltd.,NOVA Gas Transmission Ltd.,[]
247,3751154,A97723-1 Eastern ML Exp._Parkway Loop Sections...,2.58,3750786,2019-01-30 16:15:18,61,locked,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",753662,TransCanada PipeLines Limited,TransCanada PipeLines Limited,[]


In [65]:
# columns  = ['pdfId', 'pdfName', 'filingId', 'totalPages', 'application_id', 'submitter', 'company', 'consultant_name']
# df1.to_csv('consultant_name_analysis.csv', encoding = 'utf-8-sig', columns = columns)